# Problem set #7
## MACS 30100 Dr. Evans
## By Cooper Nederhood

### 1. Classifier "horse" race

In [44]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

path = "../../../ProblemSets/PS7"
file = "Auto.csv"

df = pd.read_csv(path+"/"+file,na_values="?")
df['mpg_high'] = (df['mpg'] >= df['mpg'].median()).astype(int)
df['const'] = 1
df.dropna(how='any', inplace=True)
x_var_names = [x for x in df.columns if 'mpg' not in x and x != 'name']
Xvars = df[x_var_names].values
yvals = df['mpg_high'].values

### (a) Fit a logistic model including a constant. Do k-folds validation with k=4 folds. Report MSE as average MSE across k=4 test sets. Report error rates for each category of mpg_high as the average error rate across k=4 test sets

In [48]:
from sklearn.model_selection import KFold

logit_mse = []
logit_predicted = np.array([])
logit_observed = np.array([])

k = 4
k_ind = 0
logit_kf = KFold(n_splits=k, shuffle=True, random_state=10) 

MSE_vec_kf = np.zeros(k)

for train_index, test_index in logit_kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    logit = LogisticRegression(solver='newton-cg')    
    logit.fit(X_train, y_train)
    
    y_pred = logit.predict(X_test)
    logit_mse.append( ((y_test - y_pred) ** 2).mean()  )
    
    # Append results
    logit_predicted = np.append(logit_predicted, y_pred)
    logit_observed = np.append(logit_observed, y_test)
    
    k_ind += 1


/usr/local/lib/python3.5/dist-packages/scipy/optimize/linesearch.py:461: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.5/dist-packages/scipy/optimize/linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


In [52]:
logit_avg_mse = np.mean(logit_mse)
logit_classification = classification_report(logit_observed, logit_predicted)
print("Logit avg MSE across k = 4 is: ", logit_avg_mse)

# Run classification report on our result sets
print(logit_classification)


Logit avg MSE across k = 4 is:  0.1020408163265306
             precision    recall  f1-score   support

        0.0       0.91      0.88      0.90       196
        1.0       0.88      0.92      0.90       196

avg / total       0.90      0.90      0.90       392



### (b) Fit RandomForestClassifier with max_features=2 out of the 7 possible features. Set n_estimators=20, boostrap=True, oob_score=True, and random_state=25. Report MSE as the MSE from the .oob_prediction object and report the error rates for each category of mpg_high from .oob_prediction

In [65]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

x_alt_names = [x for x in x_var_names if x != 'const']
Xvars_no_const = df[x_alt_names].values

rand_forest = RandomForestRegressor(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
rand_forest.fit(Xvars_no_const, yvals)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=True, random_state=25,
           verbose=0, warm_start=False)

In [66]:
rand_forest.score(Xvars_no_const, yvals)
rand_forest_pred = rand_for.oob_prediction_
rand_forest_mse = mean_squared_error(yvals, rand_forest_pred)
print("Random Forest MSE is: ", rand_forest_mse)

Random Forest MSE is:  0.058269344926686846


### (c) Fit a SVM with radial basis function kernel (kernel = 'rbf'), C=1, and gamma=0.2. Repeat k-fold cross validation as in part a. Report MSE and error rates across k=4 test sets

In [70]:
import numpy as np
from sklearn import svm, datasets

svm_mse = []
svm_predicted = np.array([])
svm_observed = np.array([])

k = 4
k_ind = 0
svm_kf = KFold(n_splits=k, shuffle=True, random_state=10) 

MSE_svm_kf = np.zeros(k)

for train_index, test_index in svm_kf.split(Xvars_no_const):
    X_train, X_test = Xvars_no_const[train_index], Xvars_no_const[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svm_auto = svm.SVC(kernel='rbf', C=1, gamma=0.2)    
    svm_auto.fit(Xvars_no_const, yvals)
    
    y_pred = svm_auto.predict(X_test)
    svm_mse.append( ((y_test - y_pred) ** 2).mean()  )
    
    # Append results
    svm_predicted = np.append(svm_predicted, y_pred)
    svm_observed = np.append(svm_observed, y_test)
    
    k_ind += 1

    

In [71]:
svm_avg_mse = np.mean(svm_mse)
svm_classification = classification_report(svm_observed, svm_predicted)
print("SVM avg MSE across k = 4 is: ", svm_avg_mse)

# Run classification report on our result sets
print(svm_classification)


SVM avg MSE across k = 4 is:  0.0
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       196
        1.0       1.00      1.00      1.00       196

avg / total       1.00      1.00      1.00       392



### (d) Which is the best predictor of mpg_high? Why?
Based on the out-of-sample predictive power of each model, the SVM is the best predictor. It has 100% precision (0% error) while the logit has 9% and 12% error and the random forest has non-perfect prediction